In [7]:
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
#machine learning and statistical methods
import statsmodels.api as sm
import math
#dataframe index manipulations
import datetime
from datetime import datetime
#selected preprocessing and evaluation methods
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import tensorflow as tf
import torch 


In [8]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
use_cuda

__CUDNN VERSION: 8700
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 3050 Laptop GPU
__CUDA Device Total Memory [GB]: 4.094164992


True

In [9]:
tf.config.list_physical_devices('GPU')


2023-08-21 17:43:50.821251: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-08-21 17:43:50.821405: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-08-21 17:43:50.821537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-21 17:43:50.821673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 Laptop GPU computeCapability: 8.6
coreClock: 1.035GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 163.94GiB/s
2023-08-21 17:43:50.821708: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-08-21 17:43:50.823013: I tensorflow/stream_executor/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

In [11]:




formtemplatescategoriesPharmacy=formtemplatescategories[formtemplatescategories["metaKeywords"]=='Pharmacy Forms']

formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formtemplates=formtemplates.drop_duplicates()
formandclone=pd.merge(formtemplates,formtemplatesclones,on="_id")
formtemplates=formtemplates.rename({"_id":"id"},axis=1)
formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
formtemplates2=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
formtemplates=df_melt._categories.apply(pd.Series) \
    .merge(formtemplates, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"id":"_id"},axis=1)
formcat=pd.concat([formtemplates2,formtemplates3])
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
 'name',
 'description',
 'metaKeywords',
 '_categories',
 '_id']]
formandclone=formandclone.loc[:,[
'_id',
 '_title',
 '_slug',
 '_description',
 '_language',
 '_featuredCategory',
 'formID',
 'form_type',
 'source',
 'date']]
formclonecategnonnanPharmacy=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.drop_duplicates(keep='first')

formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.date
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_period("D")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.rename({"date":"date2"},axis=1)
formclonecategnonnanPharmacy["day"]=formclonecategnonnanPharmacy.index.dayofweek
formclonecategnonnanPharmacy["week"]=formclonecategnonnanPharmacy.index.week
formclonecategnonnanPharmacy["dayofyear"]=formclonecategnonnanPharmacy.index.dayofyear
formclonecategnonnanPharmacy["year"]=formclonecategnonnanPharmacy.index.year
formclonecategnonnanPharmacy["quarter"]=formclonecategnonnanPharmacy.index.quarter
formclonecategnonnanPharmacy["month"]=formclonecategnonnanPharmacy.index.month
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_timestamp()






In [12]:
df=psql.sqldf("select count(_id),date,year,dayofyear from formclonecategnonnanPharmacy  group by date")

df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])


df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]





In [13]:
df=df.loc["01-05-2021":,]

In [14]:
import modeltrainer



In [15]:
model=modeltrainer.model_trainer(df,"PharmacyForms")

In [16]:
model.rolling()

17:44:21 - cmdstanpy - INFO - Chain [1] start processing


17:44:21 - cmdstanpy - INFO - Chain [1] done processing
17:44:21 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
17:44:21 - cmdstanpy - INFO - Chain [1] start processing


rolling prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': True}


17:44:22 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

17:44:22 - cmdstanpy - INFO - Chain [1] start processing
17:44:22 - cmdstanpy - INFO - Chain [1] done processing
17:44:22 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
17:44:22 - cmdstanpy - INFO - Chain [1] start processing


rolling prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': False}


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 19it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)



In [17]:
model.rolling_gaussian()

17:48:22 - cmdstanpy - INFO - Chain [1] start processing
17:48:22 - cmdstanpy - INFO - Chain [1] done processing
17:48:22 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
17:48:22 - cmdstanpy - INFO - Chain [1] start processing


rolling-gaussian prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': True}


17:48:23 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

17:48:23 - cmdstanpy - INFO - Chain [1] start processing
17:48:23 - cmdstanpy - INFO - Chain [1] done processing
17:48:23 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
17:48:23 - cmdstanpy - INFO - Chain [1] start processing


rolling-gaussian prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': False}


17:48:24 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

17:48:24 - cmdstanpy - INFO - Chain [1] start processing
17:48:24 - cmdstanpy - INFO - Chain [1] done processing
17:48:24 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
17:48:24 - cmdstanpy - INFO - Chain [1] start processing


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 19it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:135: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)



In [18]:
model.Nosmooth()

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:1124: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  self.history_dates = pd.to_datetime(pd.Series(df['ds'].unique(), name='ds')).sort_values()

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:271: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['ds'] = pd.to_datetime(df['ds'])

17:52:16 - cmdstanpy - INFO - Chain [1] start processing
17:52:16 - cmdstanpy - INFO - Chain [1] done processing
17:52:16 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimizat

resample prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': True}


17:52:18 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:271: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['ds'] = pd.to_datetime(df['ds'])

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:166: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:1124: UserWarning: Parsing dates in DD/MM/YYYY form

resample prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': False}


17:52:19 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:271: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['ds'] = pd.to_datetime(df['ds'])

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:166: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/prophet/forecaster.py:1124: UserWarning: Parsing dates in DD/MM/YYYY form

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")



AttributeError: Can't pickle local object 'EvaluationLoop.advance.<locals>.batch_to_device'

In [ ]:
model.exponential3()

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:187: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  _index = to_datetime(index)

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)

WA

exponential3 prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': True}
exponential3 prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'additive', 'weekly_seasonality': False}


14:35:56 - cmdstanpy - INFO - Chain [1] start processing
14:35:56 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

14:35:56 - cmdstanpy - INFO - Chain [1] start processing
14:35:56 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"

exponential3 prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'multiplicative', 'weekly_seasonality': True}
exponential3 prophet {'changepoint_prior_scale': 0.001, 'seasonality_mode': 'multiplicative', 'weekly_seasonality': False}


14:35:56 - cmdstanpy - INFO - Chain [1] start processing
14:35:56 - cmdstanpy - INFO - Chain [1] done processing
WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)

14:35:56 - cmdstanpy - INFO - Chain [1] start processing
14:35:56 - cmdstanpy - INFO - Chain [1] done processing


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 19it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/batuhan-saylam/Desktop/JotformProject/projects/modeltrainer.py:234: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)



In [ ]:
model.picking("PharmacyForms")

In [ ]:
model.model_parameters.loc[model.model_parameters["Method"]=="NeuralProphet",]

,Smoothing,Method,MAPE,Parameters,Number
13,rolling,NeuralProphet,44.749414,"{'learning_rate': 1, 'seasonality_mode': 'mult...",14
14,rolling,NeuralProphet,5.774636,"{'learning_rate': 1, 'seasonality_mode': 'mult...",15
15,rolling,NeuralProphet,162.539568,"{'learning_rate': 1, 'seasonality_mode': 'mult...",16
16,rolling,NeuralProphet,34.121338,"{'learning_rate': 1, 'seasonality_mode': 'mult...",17
17,rolling,NeuralProphet,2.399229,"{'learning_rate': 1, 'seasonality_mode': 'mult...",18
...,...,...,...,...,...
479,exponential3,NeuralProphet,36.944750,"{'learning_rate': 2, 'seasonality_mode': 'addi...",480
480,exponential3,NeuralProphet,33.031619,"{'learning_rate': 2, 'seasonality_mode': 'addi...",481
481,exponential3,NeuralProphet,24.444566,"{'learning_rate': 2, 'seasonality_mode': 'addi...",482
482,exponential3,NeuralProphet,40.472187,"{'learning_rate': 2, 'seasonality_mode': 'addi...",483


In [ ]:
model.minmape

,Smoothing,Method,MAPE,Parameters,Number
242,Nosmooth,classical,0.521949,No parameter,243


In [ ]:
model.model_parameters.loc[338,"Parameters"]

{'learning_rate': 2,
 'seasonality_mode': 'multiplicative',
 'batch_size': 16,
 'n_changepoints': 10,
 'weekly_seasonality': False,
 'epoch': 6}